# A/B Testing Final Project

Testing out the change made to the fictional "Audacity" educational website where a change was made to ask them how much time they can devote to a course before they chose to begin a free trial. 

## Experiment overview [[1]](https://docs.google.com/document/u/1/d/1aCquhIqsUApgsxQ8-SQBAigFDcfWVVohLEXcV6jWbdI/pub?embedded=True)

overview page: "start free trial", and "access course materials". If the student clicks "start free
trial", they will be asked to enter their credit card information, and then they will be enrolled in a
free trial for the paid version of the course. After 14 days, they will automatically be charged
unless they cancel first. If the student clicks "access course materials", they will be able to view
the videos and take the quizzes for free, but they will not receive coaching support or a verified
certificate, and they will not submit their final project for feedback.
In the experiment, Udacity tested a change where if the student clicked "start free trial", they
were asked how much time they had available to devote to the course. If the student indicated 5
or more hours per week, they would be taken through the checkout process as usual. If they
indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses
usually require a greater time commitment for successful completion, and suggesting that the
student might like to access the course materials for free. At this point, the student would have
the option to continue enrolling in the free trial, or access the course materials for free
instead. This screenshot shows what the experiment looks like.
The hypothesis was that this might set clearer expectations for students upfront, thus reducing
the number of frustrated students who left the free trial because they didn't have enough time—
without significantly reducing the number of students to continue past the free trial and
eventually complete the course. If this hypothesis held true, Udacity could improve the overall
student experience and improve coaches' capacity to support students who are likely to
complete the course.
The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked
by user-id from that point forward. The same user-id cannot enroll in the free trial twice. For
users that do not enroll, their user-id is not tracked in the experiment, even if they were signed
in when they visited the course overview page. 

Getting started by importing required libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
baseline = pd.read_csv('Data/baseline.csv')

In [3]:
baseline

MeasureName         Value
0  Unique cookies to view course overview page pe...  40000.000000
1   Unique cookies to click Start free trial per day   3200.000000
2                                Enrollments per day    660.000000
3      Click-through-probability on Start free trial      0.080000
4               Probability of enrolling given click      0.206250
5                Probability of payment given enroll      0.530000
6                 Probability of payment given click      0.109313

In [4]:
baseline.dtypes

MeasureName     object
 Value         float64
dtype: object

## Section 1 Question: Metric Choice

Which of the following metrics would you choose to measure for this experiment and why? For each metric you choose, indicate whether you would use it as an invariant metric or an evaluation metric. The practical significance boundary for each metric, that is, the difference that would have to be observed before that was a meaningful change for the business, is given in parentheses. All practical significance boundaries are given as absolute changes.


Any place "unique cookies" are mentioned, the uniqueness is determined by day. (That is, the same cookie visiting on different days would be counted twice.) User-ids are automatically unique since the site does not allow the same user-id to enroll twice.

Number of cookies: That is, number of unique cookies to view the course overview page. (dmin=3000)<br><br>
Number of user-ids: That is, number of users who enroll in the free trial. (dmin=50)<br><br>
Number of clicks: That is, number of unique cookies to click the "Start free trial" button (which happens before the free trial screener is trigger). (dmin=240)<br><br>
Click-through-probability: That is, number of unique cookies to click the "Start free trial" button divided by number of unique cookies to view the course overview page. (dmin=0.01)<br><br>
Gross conversion: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. (dmin= 0.01)<br><br>
Retention: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout. (dmin=0.01)<br><br>
Net conversion: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button. (dmin= 0.0075)<br><br>

You should also decide now what results you will be looking for in order to launch the experiment. Would a change in any one of your evaluation metrics be sufficient? Would you want to see multiple metrics all move or not move at the same time in order to launch? This decision will inform your choices while designing the experiment.

## Section 1 Answers: Chosing invariant and evaluation metrics

#### 1. Invarant Metrics
of the 6 metrics provided, the metrics **"Unique cookies to visit the page per day"**, **"Unique cookies to click start free trial per day"** & **"Click through probability on the start free trial"** are 3 measured before the user actually sees any changes to the page. Therefore these 2 values are not going to change as an after effect of this experiment

#### 2. Evaluation Metrics
There are 3 possible candidates for the evaluation metrics, **"Probability of enrolling given click"**, **"Probability of payment given enroll"** and **"Probability of payment given click"** are good candidates for evaluation metrics. Once I completed the sizing section below, I found that the number of PVs that would be needed in order to reliably estimate the effect of the change would be very large for "Probability of payment given enroll" therefore the final 2 metrics to measure this change are:
1. **"Probability of enrolling given click"**
2. **"Probability of payment given click"** 

### Measuring Variability
[This spreadsheet](https://www.google.com/url?q=https://docs.google.com/a/knowlabs.com/spreadsheets/d/1MYNUtC47Pg8hdoCjOXaHqF-thheGpUshrFA21BAJnNc/edit%23gid%3D0&sa=D&ust=1521269700957000&usg=AFQjCNHpp0jBBaRuv1Hm6K9O5QCC2MwAcw) contains rough estimates of the baseline values for these metrics (again, these numbers have been changed from Udacity's true numbers).


For each metric you selected as an evaluation metric, estimate its standard deviation analytically. Do you expect the analytic estimates to be accurate? That is, for which metrics, if any, would you want to collect an empirical estimate of the variability if you had time?

**Based on the question provided on the project, at first I will be calculating the baseline metrics for the evaluation metrics and their SEs using 5000 unique cookies as my unit of diversion**

In [5]:
baseline_piv = baseline.pivot_table(columns='MeasureName', aggfunc='sum')
baseline_piv.columns = ['CTR', 'Enrollments', 'P_enroll_click', 'P_pay_click', 'P_pay_enroll', 'StartTrial', 'PVs']

In [6]:
baseline_piv = baseline_piv.append({'CTR':np.nan,
                     'Enrollments':np.nan,
                     'P_enroll_click':np.nan,
                     'P_pay_click':np.nan,
                     'P_pay_enroll':np.nan,
                     'StartTrial':np.nan,
                     'PVs': 5000}, ignore_index=True)

In [7]:
baseline_piv.loc[1, 'StartTrial'] = (baseline_piv.loc[0, 'StartTrial'] *\
                                      baseline_piv.loc[1, 'PVs'])/ baseline_piv.loc[0,'PVs']
baseline_piv.loc[1, 'Enrollments'] = (baseline_piv.loc[0, 'Enrollments'] *\
                                      baseline_piv.loc[1, 'StartTrial']) \
/ baseline_piv.loc[0,'StartTrial']
baseline_piv.loc[1, 'P_pay_enroll'] = baseline_piv.loc[0, 'P_pay_enroll']
baseline_piv.loc[1, 'P_enroll_click'] = baseline_piv.loc[0, 'P_enroll_click']
baseline_piv.loc[1, 'P_pay_click'] = baseline_piv.loc[0, 'P_pay_click']
baseline_piv.loc[1, 'CTR'] = baseline_piv.loc[1, 'StartTrial'] / baseline_piv.loc[1, 'PVs']

In [8]:
baseline_piv

CTR  Enrollments  P_enroll_click  P_pay_click  P_pay_enroll  StartTrial  \
0  0.08        660.0         0.20625     0.109313          0.53      3200.0   
1  0.08         82.5         0.20625     0.109313          0.53       400.0   

       PVs  
0  40000.0  
1   5000.0

In [9]:
# function to calculate 
def calc_se(prob, n1, n2=0):
    if n2 == 0:
        se = np.sqrt(prob*(1-prob)*(1/n1))
    else:
        se = np.sqrt(prob*(1-prob)*(1/n1+1/n2))
    return round(se,4)

In [10]:
print('P_enroll_click baseline SE using unique cookies as unit of diversion: ',\
      calc_se(baseline_piv.loc[1, 'P_enroll_click'],\
              baseline_piv.loc[1, 'StartTrial']))
print('P_pay_click baseline SE using unique cookies as unit of diversion: ',\
     calc_se(baseline_piv.loc[1, 'P_pay_click'],\
              baseline_piv.loc[1, 'StartTrial']))

P_enroll_click baseline SE using unique cookies as unit of diversion:  0.0202
P_pay_click baseline SE using unique cookies as unit of diversion:  0.0156


## Section 2 Questions: Sizing
#### Choosing Number of Samples given Power
Using the analytic estimates of variance, how many pageviews total (across both groups) would you need to collect to adequately power the experiment? Use an alpha of 0.05 and a beta of 0.2. Make sure you have enough power for each metric.

Using the pageview calculator, [here](https://www.evanmiller.org/ab-testing/sample-size.html), for probability of payment given click we get the number of pageviews needed as: 819 Pvs with clicks using this click rate, total pageviews per variation are:

In [26]:
33020/0.08 * 2

825500.0

#### Choosing Duration vs. Exposure
What percentage of Udacity's traffic would you divert to this experiment (assuming there were no other experiments you wanted to run simultaneously)? Is the change risky enough that you wouldn't want to run on all traffic?


Given the percentage you chose, how long would the experiment take to run, using the analytic estimates of variance? If the answer is longer than a few weeks, then this is unreasonably long, and you should reconsider an earlier decision.

Assuming we show this change to 90% of the traffic we will need to run this experiment for:

In [27]:
825500/ (0.9*40000)

22.930555555555557

roughly 23 days

## Section 3: Analyzing results

### Source:
[[1]](https://docs.google.com/document/u/1/d/1aCquhIqsUApgsxQ8-SQBAigFDcfWVVohLEXcV6jWbdI/pub?embedded=True) - Udacity project description document